In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor


In [3]:
data = pd.read_csv('CC2020_train_final.csv')

In [4]:
data.head(3)

,YEAR,LOCATION,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,18,Loc 5608,Inbred_142,Cluster11,Tester_740,Cluster10,1.135462
1,18,Loc 4533,Inbred_142,Cluster11,Tester_740,Cluster10,1.139813
2,18,Loc 5620,Inbred_142,Cluster11,Tester_740,Cluster10,1.117778


In [5]:
X = data.drop(['YEAR', 'LOCATION'], axis=1)

In [6]:
len(X)

199476

In [7]:
clean_data = X.groupby(['INBRED', 'INBRED_CLUSTER','TESTER', 'TESTER_CLUSTER' ],as_index=False).mean()

In [8]:
clean_data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,Inbred_1071,Cluster8,Tester_1345,Cluster1,0.986544
1,Inbred_1071,Cluster8,Tester_4373,Cluster3,1.057704
2,Inbred_1071,Cluster8,Tester_4473,Cluster5,1.023704
3,Inbred_1071,Cluster8,Tester_4541,Cluster4,1.014735
4,Inbred_1071,Cluster8,Tester_5305,Cluster4,1.062727


In [9]:
len(clean_data)

10919

In [10]:
y = clean_data.YIELD

In [11]:
data = clean_data.drop(['YIELD'], axis=1)

In [12]:
data.head(5)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,Inbred_1071,Cluster8,Tester_1345,Cluster1
1,Inbred_1071,Cluster8,Tester_4373,Cluster3
2,Inbred_1071,Cluster8,Tester_4473,Cluster5
3,Inbred_1071,Cluster8,Tester_4541,Cluster4
4,Inbred_1071,Cluster8,Tester_5305,Cluster4


In [13]:
X = data

In [14]:
label_enc = preprocessing.LabelEncoder()

In [15]:
X['INBRED'] = label_enc.fit_transform(X['INBRED'])
X['INBRED_CLUSTER'] = label_enc.fit_transform(X['INBRED_CLUSTER'])
X['TESTER'] = label_enc.fit_transform(X['TESTER'])
X['TESTER_CLUSTER'] = label_enc.fit_transform(X['TESTER_CLUSTER'])

In [16]:
X.head(3)

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER
0,0,12,0,0
1,0,12,125,7
2,0,12,136,9


In [17]:
#notice how these are all same.. number varies within same location or year

In [18]:
enc = preprocessing.OneHotEncoder()

In [19]:
X = enc.fit_transform(X)

#Classifiers

In [20]:
#GradientBoosting Basic

In [21]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

loss = ['linear', 'square', 'exponential']

learning_rate = [0.05, 0.5, 1]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'loss': loss,
               'learning_rate': learning_rate
              }
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'loss': ['linear', 'square', 'exponential'], 'learning_rate': [0.05, 0.5, 1]}


In [23]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
abr = AdaBoostRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
abr_random = RandomizedSearchCV(estimator = abr, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
abr_random.fit(X, y)

Fitting 3 folds for each of 90 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  3.0min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=AdaBoostRegressor(base_estimator=None,
                                               learning_rate=1.0, loss='linear',
                                               n_estimators=50,
                                               random_state=None),
                   iid='deprecated', n_iter=100, n_jobs=-1,
                   param_distributions={'learning_rate': [0.05, 0.5, 1],
                                        'loss': ['linear', 'square',
                                                 'exponential'],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [25]:
abr_random.best_params_

{'n_estimators': 200, 'loss': 'exponential', 'learning_rate': 0.05}

In [37]:
scores = []

for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=np.random)
    my_model = AdaBoostRegressor(n_estimators = 200, loss = 'exponential', learning_rate = 0.009)
    my_model.fit(X_train, y_train)
    predictions = my_model.predict(X_test)
    score = mean_absolute_error(predictions, y_test)
    scores.append(score)

    
print('Total CV MSE is {}'.format(np.mean(scores)))
print('Total STD is {}'.format(np.std(scores)))

Total CV MSE is 0.04464019813888341
Total STD is 0.0004674312856532339
